In [69]:
import random 
import numpy as np
import torch


In [70]:
def Loss(Y,beta,X,lamd,p,K):
    sum=0
    
    for k in range(K):
        sum_betaX=0

        for j in range(p):
            sum_betaX=sum_betaX+beta[j,k]*X[j,k,:]

        sum=sum+0.5*torch.norm(Y[k,:].flatten()-sum_betaX.flatten())**2
    
    norm_infinite,indice=torch.max(torch.abs(beta),1)
    sum=sum+lamd*torch.sum(norm_infinite)

    return sum


In [71]:
#initialize settings
p=10
K=5
n_k=60
lamd=1
#guarantee each X's norm is 1
X=torch.randn(p,K,n_k)
for j in range(p):
    for k in range(K):
        X[j,k,:]=X[j,k,:]/torch.norm(X[j,k,:]) 

beta_true=torch.randint(low=-5,high=5,size=[p,K])   

#calculate Y w.r.t noise
Y=torch.zeros(K,n_k)
for k in range(K):
    sum=0
    for j in range(p):
        sum=sum+beta_true[j,k]*X[j,k,:]
    Y[k,:]=sum+torch.tensor(np.random.normal(loc=0,scale=0,size=X[j,k,:].size())).float()


In [72]:
#C_ij
C=torch.zeros(p,K)
for j in range(p):
    for k in range(K):
        C[j,k]=torch.matmul(X[j,k,:].flatten(),Y[k,:].flatten())
       
#D_ij^k
D=torch.zeros(p,p,K)
for k in range(K):
    for i in range(p):
        for j in range(p):
            D[i,j,k]=torch.matmul(X[i,k,:].flatten(),X[j,k,:].flatten())


In [73]:
#initialize beta and calculate loss
beta=torch.tensor(np.random.normal(loc=0,scale=25,size=beta_true.size())).float()
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)

tensor(12343.5205)


In [74]:
#operator for each block
def single_blcok(beta,p,K,lamd,X,Y,C,D,j_round):
    
    alpha=beta
    for k in range(K):
        beta[j_round,k]=C[j_round,k]-torch.matmul(beta[:,k].flatten().float(),D[:,j_round,k].flatten().float())+beta[j_round,k]*D[j_round,j_round,k]

    if (lamd!=0):
        if (torch.sum(torch.abs(alpha[j_round,:]))<=lamd):
            beta[j_round,:]=0
    
        alpha_j=alpha[j_round,:]
        alpha_j_abs=torch.abs(alpha_j)
        alpha_sorted,index1=torch.sort(alpha_j_abs,descending=True)
        alpha_j_sortred=torch.index_select(alpha_j,0,index1)
        judge=torch.zeros(size=alpha_j_abs.size())
    
        for l in range(K):
            judge[l]=(torch.sum(alpha_j_abs[0:l+1])-lamd)/(l+1)
    
        max_index=torch.argmax(judge)
    
    
        for k in range(max_index):
            beta[j_round,k]=torch.sign(alpha_j_sortred[k])/(max_index+1)*(torch.sum(alpha_j_abs[0:max_index+1])-lamd)

        for k in range(max_index,K):
            beta[j_round,k]=alpha_j_sortred[k]

        b,index2=torch.sort(index1)
    
        beta[j_round,:]=torch.index_select(beta[j_round,:],0,index2)
   
    return beta

In [75]:
delta_loss=100
while(delta_loss>=0.1):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

print("%f",beta)
print("%f",beta_true)

tensor(705.4390)
tensor(91.7125)
tensor(58.2465)
tensor(51.8501)
tensor(51.6495)
tensor(51.5752)
%f tensor([[-4.5763, -4.6161, -4.5763,  4.3063, -1.1352],
        [-1.9622,  1.9622, -0.2651, -1.9622, -1.9622],
        [-3.4638,  1.9726, -3.4638,  3.4638,  1.1844],
        [ 0.5130, -0.0171,  0.5130, -0.5130,  0.5130],
        [ 2.9913,  2.9913, -1.9746,  2.9913,  2.9913],
        [ 1.7899,  1.8830, -2.2600,  0.4512, -2.2600],
        [-3.3614, -3.5382,  1.6277,  3.5621,  3.9829],
        [-2.5973, -3.2044, -3.2044, -1.6488, -3.3816],
        [-2.3800, -1.2262, -2.3800,  2.3800, -2.3800],
        [-4.0316,  4.2347,  0.7068, -0.8008, -1.6491]])
%f tensor([[-5, -5, -5,  4, -1],
        [ 0,  1,  0, -5, -4],
        [-4,  2, -4,  4,  1],
        [ 0,  0,  0, -3,  0],
        [ 2,  3, -2,  4,  4],
        [ 2,  2, -4,  1, -3],
        [-3, -5,  2,  4,  4],
        [-2, -4, -4, -2, -3],
        [-1, -1, -3,  2, -5],
        [-5,  4,  1, -1, -2]])


In [76]:
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)


tensor(51.5752)


In [77]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.01):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(34.7340)
tensor(34.7991)
tensor(34.5307)
tensor(34.6821)
tensor(34.7308)
tensor(34.7420)
tensor(34.7440)


In [78]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.001):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(26.1333)
tensor(26.1740)
tensor(26.1793)
tensor(26.1801)


In [79]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.1):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(21.7436)
tensor(21.7465)


In [80]:
print("%f",beta)
print("%f",beta_true)

%f tensor([[-5.2815, -4.6047, -5.0564,  4.2948, -1.1585],
        [ 0.0139,  2.1390, -2.1390, -2.1390, -2.1390],
        [-3.6839,  1.9724, -3.6839,  3.6839,  1.1959],
        [-0.1457, -0.0223,  0.8376, -0.8376,  0.8376],
        [ 3.1888,  3.1888, -2.2216,  3.1888,  3.1888],
        [ 1.7509,  1.8871, -2.4215,  0.5126, -2.4215],
        [-2.6366, -3.5898,  1.1301,  3.6006,  3.9763],
        [-2.3380, -3.3363, -3.3363, -1.6959, -3.3321],
        [-2.5775, -1.2053, -2.5775,  2.5775, -2.5775],
        [-4.9547,  4.2066,  0.1398, -0.7992, -1.6710]])
%f tensor([[-5, -5, -5,  4, -1],
        [ 0,  1,  0, -5, -4],
        [-4,  2, -4,  4,  1],
        [ 0,  0,  0, -3,  0],
        [ 2,  3, -2,  4,  4],
        [ 2,  2, -4,  1, -3],
        [-3, -5,  2,  4,  4],
        [-2, -4, -4, -2, -3],
        [-1, -1, -3,  2, -5],
        [-5,  4,  1, -1, -2]])
